In [1]:
import src._init_paths
import src.lib.datasets.dataset.jde as datasets
import pandas as pd
import json
import matplotlib.pyplot as plt
import mmcv
import cv2
from collections import Counter
import numpy as np
import lap
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

ALLOWED = [list(range(0,6)), list(range(10, 16)),
     list(range(20, 26)),list(range(30, 36)),
     list(range(40, 46)),list(range(50, 56)),]

ALLOWED = list(np.array(ALLOWED).flatten())
ALLOWED = [str(x) for x in ALLOWED]
ALLOWED.append('00')
ALLOWED = set(ALLOWED)


In [80]:
input_video.split(',')

['/home/ubuntu/oljike/data/demo_vids/2021_01_20_Colorado_at_Washington/2021_01_20_Colorado_at_Washington.mp4']

In [79]:
input_video = '/home/ubuntu/oljike/data/demo_vids/2021_01_20_Colorado_at_Washington/2021_01_20_Colorado_at_Washington.mp4'
               
output_video = './demos/custom_5vals_colors_all_30ep_data2_50ep/2021_01_20_Colorado_at_Washington_raw.mp4'

In [3]:
ocr = json.load(open('/home/ubuntu/oljike/data/demo_vids/2021_01_20_Colorado_at_Washington/2021_01_20_Colorado_at_Washington_ocr.json'))
results = json.load(open('./demos/custom_5vals_colors_all_30ep_data2_50ep/2021_01_20_Colorado_at_Washington_raw.json'))
dataloader = datasets.LoadVideo(input_video, (1088, 608))

valid_frames = set()
for i in range(len(dataloader)):
    curr_data = ocr['results'][str(i)]
    if curr_data['score_bug_present'] and curr_data['game_clock_running']:
        valid_frames.add(i)     

Lenth of the video: 172920 frames


In [38]:
def get_results(dataloader, results):
    
    all_ids = []
    for i in range(len(dataloader)):
        if str(i) not in results:
            all_ids.append([])
        else:
            curr_hist = []
            for x in results[str(i)]:
                line = x.split(',')
                curr_hist.append(line[0])
            all_ids.append(curr_hist)
            
    tlwhs = []
    for i in range(len(dataloader)):
        if str(i) not in results:
            tlwhs.append([])
        else:
            curr_hist = []
            for x in results[str(i)]:
                line = x.split(',')
                curr_hist.append(line[1:5])
            tlwhs.append(curr_hist) 
            
            
    all_hists = []
    for i in range(len(dataloader)):
        if str(i) not in results:
            all_hists.append([])
        else:
            curr_hist = []
            for x in results[str(i)]:
                line = x.split(',')
                curr_hist.append(line[-2])
            all_hists.append(curr_hist)
            
    all_jerseys = []
    for i in range(len(dataloader)):
        if str(i) not in results:
            all_jerseys.append([])
        else:
            curr_jer = []
            for x in results[str(i)]:
                line = x.split(',')
                curr_jer.append(line[-1])

            all_jerseys.append(curr_jer)
            
    return all_ids, tlwhs, all_hists, all_jerseys

all_ids, tlwhs, all_hists, all_jerseys = get_results(dataloader, results)

In [62]:
def post_proc(all_ids, all_hists):
    
    from collections import Counter
    id_to_cls_list = {}
    for en, track_ids in enumerate(all_ids):

        for track_id, cls in zip(track_ids, all_hists[en]):
            if track_id in id_to_cls_list:
                id_to_cls_list[track_id].append(cls)
            else:
                id_to_cls_list[track_id] = [cls]

    most_commons = []
    sec_cmns = []
    len_tr = []
    id_to_cls_val = {}
    for track_id, cls_lst in id_to_cls_list.items():
        cls_lst = np.array(cls_lst).flatten().tolist()
        cnt = Counter(cls_lst)
        mst_cmn = cnt.most_common()
        cmn_1st = mst_cmn[0][0]
        if track_id=='3': 
            global xxx, cnt_xxx
            xxx = cls_lst
            cnt_xxx = mst_cmn
#             print('scnd ', cls_lst, mst_cmn)
        
        
        most_commons.append(cmn_1st)
        len_tr.append(len(cls_lst))
        if len(mst_cmn)>1:
            sec_cmns.append(mst_cmn[1][0])
        
        
        if cmn_1st is None or cmn_1st=='None':
#             print('1st NONE')
            if len(mst_cmn)>1:
                cmn_2nd = mst_cmn[1]
#                 if cmn_2nd[1]>int(0.05*len(cls_lst)) and str(cmn_2nd[0]) in ALLOWED:
                if str(cmn_2nd[0]) in ALLOWED:
#                     print("ALL ", cmn_2nd[0])
                    id_to_cls_val[track_id] = str(cmn_2nd[0])
                else:
#                     print("NA")
                    id_to_cls_val[track_id] = 'None'
            else:
                id_to_cls_val[track_id] = 'None'
        else:
            if str(cmn_1st) not in ALLOWED:
#                 print("NA2 ", type(cmn_1st), cmn_1st)
                id_to_cls_val[track_id] = 'None'
            else:
#                 print("ALL2 ", cmn_1st)
                id_to_cls_val[track_id] = str(cmn_1st)
    
    print(id_to_cls_val['3'])
    output = []
    for en, track_ids in enumerate(all_ids):
        curr_output = []
        for j in range(len(track_ids)):
            curr_output.append(id_to_cls_val[track_ids[j]])

        output.append(curr_output)
    
    print(Counter(most_commons), Counter(sec_cmns))
    return output

all_jerseys_proc = post_proc(all_ids, all_jerseys)

55
Counter({'None': 68, '5': 3, '25': 2, '24': 1, '4': 1, '21': 1, '1': 1}) Counter({'1': 10, '2': 9, '11': 8, '5': 7, 'None': 7, '21': 5, '10': 5, '0': 3, '4': 3, '55': 2, '13': 2, '25': 1, '3': 1, '41': 1, '12': 1, '15': 1})


In [70]:
for x in cnt_xxx:
    print(x,(x[1] / len(xxx))*100)

('None', 551) 34.4159900062461
('55', 333) 20.799500312304808
('25', 285) 17.801374141161773
('24', 157) 9.806371018113678
('5', 91) 5.683947532792005
('2', 41) 2.5608994378513428
('35', 26) 1.6239850093691444
('1', 22) 1.3741411617738912
('4', 15) 0.9369144284821986
('3', 14) 0.8744534665833853
('0', 12) 0.749531542785759
('15', 8) 0.49968769519050593
('21', 6) 0.3747657713928795
('11', 6) 0.3747657713928795
('13', 5) 0.3123048094940662
('45', 5) 0.3123048094940662
('22', 4) 0.24984384759525297
('10', 3) 0.18738288569643974
('14', 3) 0.18738288569643974
('30', 3) 0.18738288569643974
('33', 3) 0.18738288569643974
('20', 2) 0.12492192379762648
('51', 2) 0.12492192379762648
('34', 1) 0.06246096189881324
('23', 1) 0.06246096189881324
('00', 1) 0.06246096189881324
('53', 1) 0.06246096189881324


In [46]:
def get_color_team(cls):
    if cls=='0.0':
        return (255,0,0)
    elif cls=='1.0':
        return (0,255,0) 
def write_vid(tlwhs, all_ids, all_hists, all_jerseys, output_video, valid_frames):

    dataloader.re_init()
    valid = 0
    frame_id = 0

    ### Write to video
    img_h, img_w = (1080, 1920)
    text_scale = max(1, 1920 / 1600.)
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'MP4V'), dataloader.frame_rate, (img_w, img_h))

    jersey = None
    ball = None
    for i, (path, img, img0) in enumerate(tqdm(dataloader)):
        if i<800: continue
        if valid >= len(results): break
        curr_data = ocr['results'][str(i)]

        if curr_data['score_bug_present'] and curr_data['game_clock_running']:

            if i in valid_frames:
                online_tlwhs = tlwhs[i]
                cls = all_hists[i]
                jersey = all_jerseys[i]
                tr_id = all_ids[i]

                for ii, tlwh in enumerate(online_tlwhs):
                    x1, y1, w, h = [float(x) for x in tlwh]
                    intbox = tuple(map(int, (x1*img_w, y1*img_h, (x1*img_w + w), (y1*img_h + h))))
                    color = get_color_team(cls[ii])
                    cv2.rectangle(img0, intbox[0:2], intbox[2:4], color=color, thickness=1)

                    if jersey:
                        jersey_id = str(jersey[ii])
                        cv2.putText(img0, jersey_id + '_' + str(tr_id[ii]), (intbox[0], intbox[1] + 30), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255),
                                thickness=1)

            valid += 1

        out.write(img0)
        frame_id += 1

    out.release()    

In [13]:
write_vid(tlwhs, all_ids, all_hists, all_jerseys, output_video, valid_frames)

In [47]:
write_vid(tlwhs, all_ids, all_hists, all_jerseys_proc, output_video.replace('_raw', '_proc'), valid_frames)